<font size="6"><center>**Alpha Research: Feature Engineering**</font>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run ../nb_config.py

running notebook configuration


In [3]:
import numpy as np
import pandas as pd


In [4]:
from src.data_api import sources, sinks
from src import utils, tags

from src import data_preparation as dataprep 

In [5]:
conf = utils.get_conf()

In [6]:
fparams = conf['func_params']

start_dt = fparams['start_dt']
end_dt = fparams['end_dt']
avol_top = float(fparams['avol_top'])
avol_window = int(fparams['avol_window'])
tau = int(fparams['target']['tau'])

val_dt = fparams['split_dt']['valid']
test_dt = fparams['split_dt']['test']

# Load Data

In [7]:
wiki_prices = sources.read_wiki_prices()
wiki_prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1999-11-18 00:00:00'), 'A') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Columns: 12 entries, open to adj_volume
dtypes: float64(12)
memory usage: 1.4+ GB


In [8]:
pd.concat([wiki_prices.head(2), wiki_prices.tail(2)], axis=0)

,,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
date,ticker,,,,,,,,,,,,
1999-11-18,A,45.5000,50.0000,40.0000,44.0000,"44,739,900.0000",0.0000,1.0000,31.0420,34.1120,27.2896,30.0186,"44,739,900.0000"
1999-11-19,A,42.9400,43.0000,39.8100,40.3800,"10,897,100.0000",0.0000,1.0000,29.2954,29.3363,27.1600,27.5489,"10,897,100.0000"
2018-03-26,ZUMZ,23.7500,24.8000,23.7000,24.6500,"375,320.0000",0.0000,1.0000,23.7500,24.8000,23.7000,24.6500,"375,320.0000"
2018-03-27,ZUMZ,24.6500,24.6500,23.3500,23.6000,"403,884.0000",0.0000,1.0000,24.6500,24.6500,23.3500,23.6000,"403,884.0000"


In [9]:
wiki_prices_meta = sources.read_wiki_meta()
wiki_prices_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6834 entries, PIH to ZYME
Columns: 6 entries, name to industry
dtypes: float64(3), object(3)
memory usage: 373.7+ KB


In [10]:
pd.concat([wiki_prices_meta.head(2), wiki_prices_meta.tail(2)], axis=0)

,name,lastsale,marketcap,ipoyear,sector,industry
ticker,,,,,,
PIH,"1347 Property Insurance Holdings, Inc.",7.2001,"43,090,000.0000","2,014.0000",Finance,Property-Casualty Insurers
PIHPP,"1347 Property Insurance Holdings, Inc.",25.6200,NaN,NaN,Finance,Property-Casualty Insurers
ZUO,"Zuora, Inc.",28.3200,"3,040,000,000.0000","2,018.0000",Technology,Computer Software: Prepackaged Software
ZYME,Zymeworks Inc.,15.3000,"486,810,000.0000","2,017.0000",Health Care,Major Pharmaceuticals


In [11]:
sp500 = sources.read_market_sp500()
sp500.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2367 entries, 2011-12-08 to 2021-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SP500   2281 non-null   float64
dtypes: float64(1)
memory usage: 37.0 KB


In [12]:
pd.concat([sp500.head(2), sp500.tail(2)], axis=0)

,SP500
date,
2011-12-08,"1,234.3500"
2011-12-09,"1,255.1900"
2020-12-31,"3,756.0700"
2021-01-01,NaN


# Data Preparation

In [13]:

min_yr = '2013'  # must be below start_dt (consdering feature windows)
select_cols = ['adj_open','adj_high','adj_low', 'adj_close', 'adj_volume']

mask_min_yr = wiki_prices.index.get_level_values(tags.DATE)>=pd.Timestamp(min_yr)

feats_and_targets = (wiki_prices
    .loc[mask_min_yr, select_cols]
    .pipe(dataprep.pl_min_avol_filter, window=avol_window, top_n=avol_top)
    .pipe(dataprep.pl_add_sector, sector_map=wiki_prices_meta['sector'])
    .pipe(dataprep.pl_mkt_returns, market_prices=sp500['SP500'], tau=tau)
    .pipe(dataprep.pl_targets, tau_fwd=tau)
    .pipe(dataprep.pl_features)
    .pipe(dataprep.pl_date_filter, start_dt=start_dt, end_dt=end_dt)
    .pipe(dataprep.pl_split, val_dt=val_dt, test_dt=test_dt)
    .filter(regex=('x_|y_|sector|SP|split'))
                    )


In [14]:
pd.concat([feats_and_targets.head(2), feats_and_targets.tail(2)], axis=0)

,,sector,SP500,y_fwd_logrets_5d,x_intrad_upshadow,x_intrad_loshadow,x_returns_1d,x_returns_5d,x_zscore_5d,x_zscore_vol_5d,x_macd_5d_vs_20d,x_rsi,x_zscore_20d,x_zscore_vol_20d,x_zscore_60d,x_zscore_vol_60d,x_bb_60d,x_macd_50d_vs_252d,x_returns_1y,x_rets5d_vol_60d,x_rets5d_vol_120d,x_mkt_dispersion,x_wday_cos,x_wday_sin,x_is_eoq,x_mkt_volat_60d,x_mkt_volat_120d,split,split_3f
date,ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,A,Capital Goods,-0.0147,0.0007,0.1264,0.1847,-0.0069,-0.0115,-1.4744,0.9398,0.2672,55.3864,-0.1928,-0.5010,0.1058,-0.8054,0.0000,0.4378,0.1129,0.0405,0.0336,0.0543,0.3090,-0.9511,False,0.0236,0.0192,1.dev,1.train
2015-01-05,A,Capital Goods,-0.0341,0.0078,0.1361,0.0972,-0.0189,-0.0365,-1.5635,1.5077,0.0948,55.6667,-0.8393,-0.0693,-0.3988,-0.4243,0.0000,0.4540,0.1001,0.0406,0.0338,0.0628,1.0000,0.0000,False,0.0241,0.0194,1.dev,1.train
2017-12-28,ZTS,Health Care,0.0011,0.0133,0.1100,0.2500,-0.0008,-0.0044,0.5586,-1.2001,0.2580,59.9842,0.6349,-1.4923,1.0820,-1.6623,0.0000,8.6920,0.3552,0.0237,0.0218,0.0547,-0.8090,-0.5878,False,0.0057,0.0068,2.test,3.test
2017-12-29,ZTS,Health Care,-0.0036,0.0301,0.2100,0.0000,-0.0048,-0.0033,-0.9547,1.3702,0.2225,65.2919,0.0366,-0.0578,0.9494,-0.5330,0.0000,8.7416,0.3557,0.0238,0.0218,0.0637,0.3090,-0.9511,False,0.0056,0.0068,2.test,3.test


In [15]:
feats_and_targets.shape

(374331, 28)

In [16]:
print(feats_and_targets['split'].value_counts(normalize=True),
      '\n\n',
      feats_and_targets['split_3f'].value_counts(normalize=True))

1.dev    0.8353
2.test   0.1647
Name: split, dtype: float64 

 1.train   0.6695
2.valid   0.1657
3.test    0.1647
Name: split_3f, dtype: float64


In [17]:
feats_and_targets.filter(regex='x_').isna().sum(axis=0) / len(feats_and_targets)

x_intrad_upshadow    0.0000
x_intrad_loshadow    0.0000
x_returns_1d         0.0008
x_returns_5d         0.0041
x_zscore_5d          0.0033
x_zscore_vol_5d      0.0034
x_macd_5d_vs_20d     0.0155
x_rsi                0.0000
x_zscore_20d         0.0156
x_zscore_vol_20d     0.0156
x_zscore_60d         0.0412
x_zscore_vol_60d     0.0412
x_bb_60d             0.0000
x_macd_50d_vs_252d   0.1427
x_returns_1y         0.1431
x_rets5d_vol_60d     0.0440
x_rets5d_vol_120d    0.0776
x_mkt_dispersion     0.0000
x_wday_cos           0.0000
x_wday_sin           0.0000
x_is_eoq             0.0000
x_mkt_volat_60d      0.0000
x_mkt_volat_120d     0.0000
dtype: float64

In [18]:
sinks.write_data(feats_and_targets, table_name='feats_and_targets', layer='interim')